# overview

This Jupyter notebook example shows how to use tools from the `flux-data-qaqc` Python package to produce "corrected" energy balance closure ratios by adjusting turblent energy fluxes in order to close the surface energy balace for a daily time series from the FLUXNET 2015 dataset. The data used herein is provided with the software package and can be downloaded [here](https://github.com/Open-ET/flux-data-qaqc/blob/master/example_data/FLX_US-AR1_FLUXNET2015_SUBSET_DD_2009-2012_1-3.xlsx).

This example is provided and can be reproduced by downloading `fluxdataqaqc` from github with git 

```bash
git clone https://github.com/Open-ET/flux-data-qaqc.git
```

or you can download a compressed folder. Dependencies currently include pandas, refet, numpy, xlrd (for reading excel input), and bokeh. Automatic installation and dependencies handling will soon be provided along with online documentation.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
# currently not installable so import from parent dir
sys.path.append('..')
from fluxdataqaqc.data import Data
from fluxdataqaqc.qaqc import QaQc 
from fluxdataqaqc.plot import Plot 
from bokeh.plotting import figure, show
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## create a ``Data`` object to read in time series data using a config file

In [2]:
config_path = 'fluxnet_config.ini'
d = Data(config_path)

In [3]:
# you can access all metadata and datain the config file as a list
d.config.items('METADATA') # can access the DATA section the same way

[('climate_file_path', 'FLX_US-AR1_FLUXNET2015_SUBSET_DD_2009-2012_1-3.xlsx'),
 ('site_id', 'US-AR1'),
 ('station_latitude', '36.4267'),
 ('station_longitude', '-99.42'),
 ('station_elevation', '611'),
 ('anemometer_height', '3'),
 ('missing_data_value', '-9999')]

In [4]:
# or as a dict, e.g. to access specific values by name
d.config.get('METADATA','station_elevation')

'611'

In [5]:
# path to climate time series input and config files
print(d.climate_file, '\n', d.config_file)

/home/john/flux-data-qaqc/examples/FLX_US-AR1_FLUXNET2015_SUBSET_DD_2009-2012_1-3.xlsx 
 /home/john/flux-data-qaqc/examples/fluxnet_config.ini


In [6]:
# view full header of input time series file
d.header

Index(['TIMESTAMP', 'TA_F', 'TA_F_QC', 'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC',
       'LW_IN_F', 'LW_IN_F_QC', 'VPD_F', 'VPD_F_QC', 'PA_F', 'PA_F_QC', 'P_F',
       'P_F_QC', 'WS_F', 'WS_F_QC', 'USTAR', 'USTAR_QC', 'NETRAD', 'NETRAD_QC',
       'PPFD_IN', 'PPFD_IN_QC', 'PPFD_OUT', 'PPFD_OUT_QC', 'SW_OUT',
       'SW_OUT_QC', 'LW_OUT', 'LW_OUT_QC', 'CO2_F_MDS', 'CO2_F_MDS_QC',
       'TS_F_MDS_1', 'TS_F_MDS_1_QC', 'SWC_F_MDS_1', 'SWC_F_MDS_1_QC',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'LE_CORR_25', 'LE_CORR_75', 'LE_RANDUNC', 'H_F_MDS', 'H_F_MDS_QC',
       'H_CORR', 'H_CORR_25', 'H_CORR_75', 'H_RANDUNC', 'NEE_VUT_REF',
       'NEE_VUT_REF_QC', 'NEE_VUT_REF_RANDUNC', 'NEE_VUT_25', 'NEE_VUT_50',
       'NEE_VUT_75', 'NEE_VUT_25_QC', 'NEE_VUT_50_QC', 'NEE_VUT_75_QC',
       'RECO_NT_VUT_REF', 'RECO_NT_VUT_25', 'RECO_NT_VUT_50', 'RECO_NT_VUT_75',
       'GPP_NT_VUT_REF', 'GPP_NT_VUT_25', 'GPP_NT_VUT_50', 'GPP_NT_VUT_75',
       'RECO_DT_VUT_REF', 'RECO_D

# load date-indexed DataFrame using ``.df``

* note, if there are variables stated in the config file but not found in the header of the input file, they will be filled with NaN (null) values in the dataframe

In [7]:
d.df.head()

,t_avg,sw_pot,sw_in,lw_in,vpd,ppt,ws,Rn,sw_out,lw_out,G,LE,LE_corr,H,H_corr
date,,,,,,,,,,,,,,,
2009-01-01,2.803,186.710,123.108,261.302,1.919,0.0,3.143,NaN,NaN,NaN,NaN,67.1459,43.8414,20.3876,13.3116
2009-01-02,2.518,187.329,121.842,268.946,0.992,0.0,2.093,NaN,NaN,NaN,NaN,92.8616,60.9673,32.6505,21.4364
2009-01-03,5.518,188.008,124.241,268.004,2.795,0.0,4.403,NaN,NaN,NaN,NaN,75.8029,50.3151,20.0569,13.3130
2009-01-04,-3.753,188.742,113.793,246.675,0.892,0.0,4.336,NaN,NaN,NaN,NaN,67.1459,45.0539,20.3876,13.6798
2009-01-05,-2.214,189.534,124.332,244.478,1.304,0.0,2.417,NaN,NaN,NaN,NaN,92.8616,62.6443,32.6505,22.0260


## you can modify the dataframe or assign new columns or even assign a new dataframe within Python

In [8]:
x = d.df
x += 100
d.df = x
d.df *= 5
d.df.head()

,t_avg,sw_pot,sw_in,lw_in,vpd,ppt,ws,Rn,sw_out,lw_out,G,LE,LE_corr,H,H_corr
date,,,,,,,,,,,,,,,
2009-01-01,514.015,1433.550,1115.540,1806.510,509.595,500.0,515.715,NaN,NaN,NaN,NaN,835.7295,719.2070,601.9380,566.558
2009-01-02,512.590,1436.645,1109.210,1844.730,504.960,500.0,510.465,NaN,NaN,NaN,NaN,964.3080,804.8365,663.2525,607.182
2009-01-03,527.590,1440.040,1121.205,1840.020,513.975,500.0,522.015,NaN,NaN,NaN,NaN,879.0145,751.5755,600.2845,566.565
2009-01-04,481.235,1443.710,1068.965,1733.375,504.460,500.0,521.680,NaN,NaN,NaN,NaN,835.7295,725.2695,601.9380,568.399
2009-01-05,488.930,1447.670,1121.660,1722.390,506.520,500.0,512.085,NaN,NaN,NaN,NaN,964.3080,813.2215,663.2525,610.130


---
# using the `QaQc` class to correct latent energy and sensible heat

* note, the method used for corrections will be documented soon

In [9]:
# read in data fresh and use it to create a QaQc instance
data = Data(config_path)
q = QaQc(data)

In [10]:
# data is not corrected yet:
q.corrected

False

In [11]:
# data has not changed...
q.df.head()

,t_avg,sw_pot,sw_in,lw_in,vpd,ppt,ws,Rn,sw_out,lw_out,G,LE,LE_corr,H,H_corr
date,,,,,,,,,,,,,,,
2009-01-01,2.803,186.710,123.108,261.302,1.919,0.0,3.143,NaN,NaN,NaN,NaN,67.1459,43.8414,20.3876,13.3116
2009-01-02,2.518,187.329,121.842,268.946,0.992,0.0,2.093,NaN,NaN,NaN,NaN,92.8616,60.9673,32.6505,21.4364
2009-01-03,5.518,188.008,124.241,268.004,2.795,0.0,4.403,NaN,NaN,NaN,NaN,75.8029,50.3151,20.0569,13.3130
2009-01-04,-3.753,188.742,113.793,246.675,0.892,0.0,4.336,NaN,NaN,NaN,NaN,67.1459,45.0539,20.3876,13.6798
2009-01-05,-2.214,189.534,124.332,244.478,1.304,0.0,2.417,NaN,NaN,NaN,NaN,92.8616,62.6443,32.6505,22.0260


In [12]:
# note the original columns
import pprint
pprint.pprint(', '.join(q.df.columns))

('t_avg, sw_pot, sw_in, lw_in, vpd, ppt, ws, Rn, sw_out, lw_out, G, LE, '
 'LE_corr, H, H_corr')


In [13]:
q.elevation, q.latitude # necessary for computing clear sky radiation

(611, 36.4267)

In [14]:
# run correction
q.correct_data()
q.df.head()

,t_avg,sw_pot,sw_in,lw_in,vpd,ppt,ws,Rn,sw_out,lw_out,...,H_adj,flux_adj,flux_corr,et_corr,ebc_corr,et_reg,et_adj,ebc_reg,ebc_adj,rso
date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,2.803,186.710,123.108,261.302,1.919,0.0,3.143,NaN,NaN,NaN,...,20.3876,87.5335,57.1530,1.515159,NaN,2.320562,2.320562,NaN,NaN,141.552135
2009-01-02,2.518,187.329,121.842,268.946,0.992,0.0,2.093,NaN,NaN,NaN,...,32.6505,125.5121,82.4037,2.107030,NaN,3.209297,3.209297,NaN,NaN,142.036533
2009-01-03,5.518,188.008,124.241,268.004,2.795,0.0,4.403,NaN,NaN,NaN,...,20.0569,95.8598,63.6281,1.738890,NaN,2.619748,2.619748,NaN,NaN,142.559581
2009-01-04,-3.753,188.742,113.793,246.675,0.892,0.0,4.336,NaN,NaN,NaN,...,20.3876,87.5335,58.7337,1.557063,NaN,2.320562,2.320562,NaN,NaN,143.121137
2009-01-05,-2.214,189.534,124.332,244.478,1.304,0.0,2.417,NaN,NaN,NaN,...,32.6505,125.5121,84.6703,2.164987,NaN,3.209297,3.209297,NaN,NaN,143.721045


# correct energy balance using `flux-data-qaqc` methods

In [15]:
q.correct_data()
q.corrected

True

In [16]:
# now we have original data plus adjusted variables, energy balance ratios, and others
pprint.pprint(', '.join(q.df.columns))

('t_avg, sw_pot, sw_in, lw_in, vpd, ppt, ws, Rn, sw_out, lw_out, G, LE, '
 'LE_corr, H, H_corr, energy, flux, bowen_ratio, LE_adj, H_adj, flux_adj, '
 'flux_corr, et_corr, ebc_corr, et_reg, et_adj, ebc_reg, ebc_adj, rso')


In [17]:
q.df.head()

,t_avg,sw_pot,sw_in,lw_in,vpd,ppt,ws,Rn,sw_out,lw_out,...,H_adj,flux_adj,flux_corr,et_corr,ebc_corr,et_reg,et_adj,ebc_reg,ebc_adj,rso
date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,2.803,186.710,123.108,261.302,1.919,0.0,3.143,NaN,NaN,NaN,...,20.3876,87.5335,57.1530,1.515159,NaN,2.320562,2.320562,NaN,NaN,141.552135
2009-01-02,2.518,187.329,121.842,268.946,0.992,0.0,2.093,NaN,NaN,NaN,...,32.6505,125.5121,82.4037,2.107030,NaN,3.209297,3.209297,NaN,NaN,142.036533
2009-01-03,5.518,188.008,124.241,268.004,2.795,0.0,4.403,NaN,NaN,NaN,...,20.0569,95.8598,63.6281,1.738890,NaN,2.619748,2.619748,NaN,NaN,142.559581
2009-01-04,-3.753,188.742,113.793,246.675,0.892,0.0,4.336,NaN,NaN,NaN,...,20.3876,87.5335,58.7337,1.557063,NaN,2.320562,2.320562,NaN,NaN,143.121137
2009-01-05,-2.214,189.534,124.332,244.478,1.304,0.0,2.417,NaN,NaN,NaN,...,32.6505,125.5121,84.6703,2.164987,NaN,3.209297,3.209297,NaN,NaN,143.721045


In [18]:
# view time series of select variable
p = figure(x_axis_label='date', y_axis_label='net radiation w/m2')
p.line(q.df.index, q.df.Rn, line_width=2)
p.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")
show(p)

## temporally aggregate to monthly data using sums for ET and P, and means for all others

In [19]:
q.monthly_df.head()

,H_adj,vpd,flux_adj,flux,LE,ebc_corr,rso,sw_in,sw_out,H_corr,...,lw_out,ws,flux_corr,energy,ebc_reg,Rn,et_reg,et_adj,ppt,et_corr
month,,,,,,,,,,,,,,,,,,,,,
2009-01-31,24.236697,1.959129,102.470565,102.470565,78.233868,NaN,154.204780,128.720355,NaN,17.626603,...,NaN,3.534355,74.692668,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-02-28,24.660911,3.757714,103.773596,103.773596,79.112686,NaN,197.213823,175.166750,NaN,19.408454,...,NaN,3.828571,82.272961,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-03-31,24.226029,6.457871,102.739155,102.739155,78.513126,NaN,256.339885,200.775032,NaN,18.794103,...,NaN,4.358710,79.708248,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-04-30,24.612837,4.942533,103.674777,103.674777,79.061940,NaN,313.753204,245.731533,39.416091,19.565137,...,438.996567,4.482000,82.409700,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-05-31,26.615866,6.900710,131.125547,131.125547,104.509681,NaN,352.136946,248.396258,43.397006,21.571916,...,425.218290,3.529613,106.351555,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## alternatively create a QaQc instance from a pandas.DataFrame using `QaQc.from_dataframe`

Be sure to have the main energy balance components in the dataframe at daily time steps the dataframe index should be a daily datetime index as well, they should be named: 
* Rn, G, H, LE  

Otherwise you will not be able to run the energy balance correction routine, the example below shows that only the four energy balance components are needed to run the routine. **Note:**  we need to assign station elevation (m) and latitude (dec. degrees) which are normaly in the config file however this method gives the ability to use arbitrary daily time series data within Python.

In [20]:
data = Data(config_path)
# using the same dataframe here but this can be any with the correct variable names
df = data.df
# drop all other variables except those needed to demonstrate
df = df.drop([c for c in df.columns if not c in ['Rn', 'G', 'H', 'LE']], axis=1)
q = QaQc.from_dataframe(df, site_id='US-AR1', elev_m=611, lat_dec_deg=36.4267)
q.df.head()

,Rn,G,LE,H
date,,,,
2009-01-01,NaN,NaN,67.1459,20.3876
2009-01-02,NaN,NaN,92.8616,32.6505
2009-01-03,NaN,NaN,75.8029,20.0569
2009-01-04,NaN,NaN,67.1459,20.3876
2009-01-05,NaN,NaN,92.8616,32.6505


## compare monthly energy balance closure ratio with raw data and corrected

In [21]:
p = figure(x_axis_label='date', y_axis_label='energy balance correction ratio')
p.line(q.monthly_df.index, q.monthly_df['ebc_reg'], color='red', legend="Raw", line_width=2)
p.line(q.monthly_df.index, q.monthly_df['ebc_adj'], legend="Corrected", line_width=2)
p.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")
show(p)

## save daily and monthly time series of input and computed variables

* Output by default is saved to *[site_id]_output* directory, in this case *US-AR1_output*
* **Note:** the `QaQc.write` method will run energy balance corrections and produce the corrected and raw versions of the energy balance closure ratio at monthly and daily frequencies if they were not previously produced, i.e. if `QaQc.correct_data` and `QaQc.monthly_df` have not yet been called.

In [22]:
q.write()

/home/john/flux-data-qaqc/examples/US-AR1_output does not exist, creating directory


In [23]:
# view contents of newly created output directory
for f in q.out_dir.glob('*'): print(f)

/home/john/flux-data-qaqc/examples/US-AR1_output/daily_data.csv
/home/john/flux-data-qaqc/examples/US-AR1_output/monthly_data.csv


as shown above the output directory is a system independent `pathlib.Path` absolute file path that is added as an instance attribute

In [24]:
q.out_dir

PosixPath('/home/john/flux-data-qaqc/examples/US-AR1_output')

alternatively you can name your own output directory and this will be passed on to `Plot` routines so that the saved plots go into the same directory:

In [25]:
q.write(out_dir='my_output_folder')
print(q.out_dir,'\n','\n'.join([str(f) for f in q.out_dir.glob('*')]))

/home/john/flux-data-qaqc/examples/my_output_folder does not exist, creating directory
/home/john/flux-data-qaqc/examples/my_output_folder 
 /home/john/flux-data-qaqc/examples/my_output_folder/daily_data.csv
/home/john/flux-data-qaqc/examples/my_output_folder/monthly_data.csv


# using the `Plot` class to create multiple comparison and time series plots for QA/QC and data validation

In [26]:
# create a Plot object
config_path = 'fluxnet_config.ini'
d = Data(config_path)
q = QaQc(d)
plt = Plot(q)

In [27]:
# generate plots
plt.generate_plots()


Vapor Pressure graph missing a variable.


../fluxdataqaqc/plot.py:331: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  slope_orig, _, _, _ = np.linalg.lstsq(var_one_lstsq, var_two_lstsq)
../fluxdataqaqc/plot.py:332: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  slope_corr, _, _, _ = np.linalg.lstsq(var_one_lstsq, var_three_lstsq)


In [28]:
# access the plot file path
plt.plot_file

PosixPath('/home/john/flux-data-qaqc/examples/US-AR1_output/plots.html')

## The plots generated by `flux-data-qaqc` are all saved to a single HTML file with [bokeh](https://bokeh.pydata.org/en/latest/) subplots

* **Note:** if variables needed for plots are missing some plots will not appear

In [29]:
# view outplot plots within Jupyter notebook
from IPython.display import HTML
HTML(filename=plt.plot_file)